In [57]:
import csv, json
import geojson
from geojson import Feature, FeatureCollection, Point, dump
import pandas as pd


In [35]:
ufo = pd.read_csv(r'UFO Files/us_ufo_csv.csv')
ufo.head()

,Unnamed: 0,datetime,city,state,country,shape,duration (seconds),latitude,longitude
0,0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,29.883056,-97.941111
1,3,10/10/1956 21:00,edna,tx,us,circle,20,28.978333,-96.645833
2,4,10/10/1960 20:00,kaneohe,hi,us,light,900,21.418056,-157.803611
3,5,10/10/1961 19:00,bristol,tn,us,sphere,300,36.595000,-82.188889
4,7,10/10/1965 23:45,norwalk,ct,us,disk,1200,41.117500,-73.408333


In [86]:
# create the function
def ufo_to_geojson(ufo, properties, latitude='latitude', longitude='longitude'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary

    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lng : the name of the column in the dataframe that contains longitude data
    """

    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    # x is the equivalent of the row, df.iterrows converts the dataframe in to a pd.series object
    # the x is a counter and has no influence
    for x, row in ufo.iterrows():

        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [float(row.longitude),float(row.latitude)]

        # be aware that the dataframe is a pd.series
        #print('rowitem converts to ndarray(numpy) :\n ', row)
        # convert the array to a pandas.serie
        geo_props = pd.Series(row)

        # for each column, get the value and add it as a new feature property
        # prop determines the list from the properties
        for prop in properties:

            #loop over the items to convert to string elements

            #convert to string
            if type(geo_props[prop]) == float:
                #print('ok')
                geo_props[prop] = str(int(geo_props[prop]))

            # now create a json format, here we have to make the dict properties
            feature['properties'][prop] = geo_props[prop]

        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
        
    return geojson

with open('ufomap.geojson','w') as f:
    json.dump(ufo_to_geojson(ufo,['latitude','longitude']),f)